In [1]:
import pandas as pd
import numpy as np
import missingno as msno

<h4>Passo 1 - Leitura do CSV</h4>

In [2]:
#LOADING DATASET
path_dataset = "./base_mensalizada_de_funcionarios.csv"

df_raw = pd.read_csv(path_dataset, sep=";")
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mes_referencia      121 non-null    object
 1   id_funcionario      121 non-null    int64 
 2   data_de_nascimento  118 non-null    object
 3   data_de_admissao    117 non-null    object
 4   data_de_demissao    2 non-null      object
 5   grau_de_instrucao   121 non-null    object
 6   cargo               121 non-null    object
 7   salario             121 non-null    object
dtypes: int64(1), object(7)
memory usage: 7.7+ KB


<h4>Passo 2.1 - Completude </h4>

In [3]:
#AJUSTING DATA TYPES TO DATA
df_raw["mes_referencia"] = pd.to_datetime(df_raw["mes_referencia"], format="%d/%m/%Y")
df_raw["data_de_nascimento"] = pd.to_datetime(df_raw["data_de_nascimento"], format="%d/%m/%Y")
df_raw["data_de_admissao"] = pd.to_datetime(df_raw["data_de_admissao"], format="%d/%m/%Y")
df_raw["data_de_demissao"] = pd.to_datetime(df_raw["data_de_demissao"],  format="%d/%m/%Y")

In [4]:
df_raw.isnull().sum()

mes_referencia          0
id_funcionario          0
data_de_nascimento      3
data_de_admissao        4
data_de_demissao      119
grau_de_instrucao       0
cargo                   0
salario                 0
dtype: int64

In [5]:
#AJUSTING SALARY TO FLOAT
for i in range(0,len(df_raw)):
    if df_raw["salario"][i].isnumeric():
        continue
    else:
        df_raw["salario"][i] = np.NaN

for func in df_raw["id_funcionario"].unique():
    df_aux = df_raw[df_raw["id_funcionario"] == func]
    df_raw["salario"][df_raw["id_funcionario"] == func] = df_aux["salario"].interpolate(method='pad', order=2)

df_raw["salario"] = df_raw["salario"].astype(float)

/tmp/ipykernel_34178/4257497274.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["salario"][i] = np.NaN
/tmp/ipykernel_34178/4257497274.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["salario"][df_raw["id_funcionario"] == func] = df_aux["salario"].interpolate(method='pad', order=2)


In [6]:
#AJUSTING DATES
for func in df_raw["id_funcionario"].unique():
    df_aux = df_raw[df_raw["id_funcionario"] == func]

    # df_raw["data_de_admissao"][df_raw["id_funcionario"] == func] = df_aux["data_de_admissao"].interpolate(method='pad', order=1)     #AJUSTING ADMINISSION DATE
    # df_raw["data_de_nascimento"][df_raw["id_funcionario"] == func] = df_aux["data_de_nascimento"].interpolate(method='pad', order=1) #AJUSTING DOB
    df_raw["data_de_admissao"][df_raw["id_funcionario"] == func] = df_aux["data_de_admissao"].bfill().ffill()
    df_raw["data_de_nascimento"][df_raw["id_funcionario"] == func] = df_aux["data_de_nascimento"].bfill().ffill()

/tmp/ipykernel_34178/2428437352.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["data_de_admissao"][df_raw["id_funcionario"] == func] = df_aux["data_de_admissao"].bfill().ffill()
/tmp/ipykernel_34178/2428437352.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw["data_de_nascimento"][df_raw["id_funcionario"] == func] = df_aux["data_de_nascimento"].bfill().ffill()


In [7]:
# df_raw[df_raw['data_de_admissao'].isnull()]
# df_raw[df_raw['data_de_nascimento'].isnull()]

In [8]:
df_raw.isnull().sum()

mes_referencia          0
id_funcionario          0
data_de_nascimento      0
data_de_admissao        0
data_de_demissao      119
grau_de_instrucao       0
cargo                   0
salario                 0
dtype: int64

<h4>Passo 2.2 - Unicidade </h4>

In [9]:
df_raw.shape

(121, 8)

In [10]:
df_raw = df_raw.loc[~df_raw[["mes_referencia", "id_funcionario"]].duplicated()]
# df_raw.reset_index

In [11]:
df_raw.shape

(115, 8)

<h4>Passo 2.3  a 2.5 - Integridade e Padrões </h4>

In [12]:
df_raw["grau_de_instrucao"] = df_raw["grau_de_instrucao"].str.capitalize()
df_raw["cargo"] = df_raw["cargo"].str.title()

df_raw["grau_de_instrucao"] = df_raw["grau_de_instrucao"].str.lstrip()
df_raw["cargo"] = df_raw["cargo"].str.lstrip()

In [28]:
# msno.bar(df_raw)
# msno.matrix(df_raw)

In [15]:
# df_raw[20:50]

<h4>Passo 3.1 - Admitido no mês</h4>

In [16]:
df_admitido_mes = df_raw[(pd.DatetimeIndex(df_raw['data_de_admissao']).month == pd.DatetimeIndex(df_raw['mes_referencia']).month) & 
                         (pd.DatetimeIndex(df_raw['data_de_admissao']).year  == pd.DatetimeIndex(df_raw['mes_referencia']).year) ].index
# df_admitido_mes = df_raw[(pd.DatetimeIndex(df_raw['data_de_admissao']).month == pd.DatetimeIndex(df_raw['mes_referencia']).month)].index

df_raw["admitido_no_mes"] = [1 if idx in df_admitido_mes else 0 for idx in df_raw.index]

<h4>Passo 3.2 - Demitido no mês</h4>

In [17]:
df_demitido_mes = df_raw[(pd.DatetimeIndex(df_raw['data_de_demissao']).month == pd.DatetimeIndex(df_raw['mes_referencia']).month) & 
                         (pd.DatetimeIndex(df_raw['data_de_demissao']).year  == pd.DatetimeIndex(df_raw['mes_referencia']).year) ].index
# df_demitido_mes = df_raw[(pd.DatetimeIndex(df_raw['data_de_demissao']).month == pd.DatetimeIndex(df_raw['mes_referencia']).month)].index

df_raw["demitido_no_mes"] = [1 if idx in df_demitido_mes else 0 for idx in df_raw.index]

<h4>Passo 3.3 - Idade</h4>

In [18]:
def calculate_year_between(start, now):
    try: 
        birthday = start.replace(year=now.year)
    except ValueError: # raised when birth date is February 29 and the current year is not a leap year
        birthday = start.replace(year=now.year, month=start.month+1, day=1)
    if birthday > now:
        return now.year - start.year - 1
    else:
        return now.year - start.year

df_raw["idade"] = [calculate_year_between(df_raw["data_de_nascimento"][idx], df_raw["mes_referencia"][idx]) for idx in df_raw.index]

<h4>Passo 3.4 - Escolaridade categorizada</h4>
<h5>Categorias:
<ul>
  <li>ensino fundamental incompleto</li>
  <li>ensino fundamental completo</li>
  <li>ensino médio incompleto</li>
  <li>ensino médio completo</li>
  <li>ensino superior incompleto</li>
  <li>ensino superior completo</li>
  <li>pós graduação</li>
  <li>mestrado</li>
  <li>doutorado</li>
</ul>
</h5>

In [19]:
categoria_grau =   ["ensino fundamental incompleto", #0
                        "ensino fundamental completo",   #1
                        "ensino médio incompleto",       #2
                        "ensino médio completo",         #3 
                        "ensino superior incompleto",    #4
                        "ensino superior completo",      #5
                        "pós graduação",                 #6
                        "mestrado",                      #7
                        "doutorado"]                     #8
    
categoria_grau = [categoria_grau[idx].title() for idx in range(0, len(categoria_grau))]

def escolaridade(grau_instrucao):
    grau_instrucao = grau_instrucao.lower()

    if "funda" in grau_instrucao or "série" in grau_instrucao :
        if "incom" in grau_instrucao or "série" in grau_instrucao:
            return categoria_grau[0]
        elif "comp" in grau_instrucao:
            return categoria_grau[1]
    elif "médio" in grau_instrucao or "medio" in grau_instrucao:
        if "incom" in grau_instrucao:
            return categoria_grau[2]
        elif "comp" in grau_instrucao:
            return categoria_grau[3]
    elif "superior" in grau_instrucao or "faculdade" in grau_instrucao or ("graduação" in grau_instrucao and not "pós" in grau_instrucao):
        if "incom" in grau_instrucao:
            return categoria_grau[4]
        elif "comp" in grau_instrucao or "graduação" in grau_instrucao:
            return categoria_grau[5]
    elif "pós graduação" in grau_instrucao:
        return categoria_grau[6]
    elif "mestrado" in grau_instrucao:
        return categoria_grau[7]
    elif "doutorado" in grau_instrucao:
        return categoria_grau[8]
    else:
        return "FORA"

df_raw["grau_de_instrucao"] = [escolaridade(df_raw["grau_de_instrucao"][idx]) for idx in df_raw.index]

<h4>Passo 3.5 - Tempo de empresa</h4>

In [20]:
df_raw["tempo_de_empresa"] = [calculate_year_between(df_raw["data_de_admissao"][idx], df_raw["mes_referencia"][idx]) for idx in df_raw.index]

<h4>Passo 3.6 - Mudou de salário</h4>

In [21]:
df_new = []
for func in df_raw["id_funcionario"].unique():
    df_aux = df_raw[df_raw["id_funcionario"] == func]

    df_roll = np.roll(df_aux["salario"], 1)
    df_roll[0] = df_roll[1]
    df_rolled = pd.DataFrame(df_roll, index=df_aux.index, columns=["salario"])

    df_new.append(pd.DataFrame([1 if df_rolled["salario"][idx] != df_aux["salario"][idx] else 0 for idx in df_aux.index], index=df_aux.index, columns=["mudou_de_salario"]))

df_raw["mudou_de_salario"] = pd.concat(df_new, join='outer', axis=0)

<h4>Passo 3.7 - Mudou de cargo</h4>

In [22]:
df_new = []
for func in df_raw["id_funcionario"].unique():
    df_aux = df_raw[df_raw["id_funcionario"] == func]

    df_roll = np.roll(df_aux["cargo"], 1)
    df_roll[0] = df_roll[1]
    df_rolled = pd.DataFrame(df_roll, index=df_aux.index, columns=["cargo"])

    df_new.append(pd.DataFrame([1 if df_rolled["cargo"][idx] != df_aux["cargo"][idx] else 0 for idx in df_aux.index], index=df_aux.index, columns=["mudou_de_cargo"]))

df_raw["mudou_de_cargo"] = pd.concat(df_new, join='outer', axis=0)

In [23]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 0 to 120
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   mes_referencia      115 non-null    datetime64[ns]
 1   id_funcionario      115 non-null    int64         
 2   data_de_nascimento  115 non-null    datetime64[ns]
 3   data_de_admissao    115 non-null    datetime64[ns]
 4   data_de_demissao    2 non-null      datetime64[ns]
 5   grau_de_instrucao   115 non-null    object        
 6   cargo               115 non-null    object        
 7   salario             115 non-null    float64       
 8   admitido_no_mes     115 non-null    int64         
 9   demitido_no_mes     115 non-null    int64         
 10  idade               115 non-null    int64         
 11  tempo_de_empresa    115 non-null    int64         
 12  mudou_de_salario    115 non-null    int64         
 13  mudou_de_cargo      115 non-null    int64         